In [1]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
from datetime import date

In [2]:
df_old = pd.read_csv("brd_covid_19.csv",parse_dates=[7])

In [3]:
df_old = df_old.sort_values(["bundesland", "datum"])
df_old.drop_duplicates(inplace=True)
df_old = df_old.reset_index().drop("index",axis = 1)

In [4]:
today = date.today()

In [5]:
URL = "https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html"

In [6]:
page = requests.get(URL)

In [7]:
soup = BeautifulSoup(page.content, 'html.parser')

In [8]:
results = soup.find_all("tr")

In [9]:
bundesland = [re.sub('<[^<]+?>', '', str(elm.find_all("td")[0])).replace("\xad","").replace("\n","") for idx, elm in enumerate(results) if idx > 1]
faelle = [re.sub('<[^<]+?>', '', str(elm.find_all("td")[1])).replace(".","") for idx, elm in enumerate(results) if idx > 1]
neue_faelle = [re.sub('<[^<]+?>', '', str(elm.find_all("td")[2])).replace(".","").strip("+").strip("*") for idx, elm in enumerate(results) if idx > 1]
faelle_per_capita = [re.sub('<[^<]+?>', '', str(elm.find_all("td")[3])).replace(".","") for idx, elm in enumerate(results) if idx > 1]
tod = [re.sub('<[^<]+?>', '', str(elm.find_all("td")[4])).replace(".","") for idx, elm in enumerate(results) if idx > 1]

In [10]:
lat={"Baden-Württemberg":48.5, "Bayern":48.9, "Berlin":52.5, "Brandenburg":52, "Bremen":53.1, "Hamburg":53.5, 
     "Hessen":50.7, "Mecklenburg-Vorpommern":53.8, "Niedersachsen":52.57, "Nordrhein-Westfalen":51.6,"Rheinland-Pfalz":50, 
     "Saarland":49.4, "Sachsen":51.05, "Sachsen-Anhalt":52, "Schleswig-Holstein":54.2, "Thüringen":50.85, "Gesamt":51.06}
long={"Baden-Württemberg":9, "Bayern":11.4, "Berlin":13.4, "Brandenburg":13.87, "Bremen":8.8, "Hamburg":10, 
      "Hessen":9, "Mecklenburg-Vorpommern":12.5,"Niedersachsen":10, "Nordrhein-Westfalen":7.5,"Rheinland-Pfalz":7.3,
      "Saarland":6.97,"Sachsen":13.5, "Sachsen-Anhalt":11.65,"Schleswig-Holstein":9.9, "Thüringen":11.13,"Gesamt":10}


In [11]:
df=pd.DataFrame(data=[bundesland,faelle,tod,neue_faelle,faelle_per_capita]).T
    

In [12]:
df["lat"]="NaN"
df["long"]="NaN"
df["date"] = today

In [13]:
df.columns = ["bundesland","faelle","tod","neue_faelle","faelle_per_capita","lat","long","datum"]

In [14]:
df.datum = pd.to_datetime(df.datum)

In [15]:
df.bundesland.replace("Schleswig Holstein","Schleswig-Holstein", inplace=True)
df.tod = df.tod.replace(to_replace = "", value = "0", regex = True)

In [16]:
df["lat"]=[lat[elm] for elm in df.bundesland.values]
df["long"]=[long[elm] for elm in df.bundesland.values]


In [17]:
#df.lat = df.lat.astype("float")
#df.long = df.long.astype("float")

df.tod = df.tod.astype("int64")
df.faelle = df.faelle.astype("int64")
df.neue_faelle = df.neue_faelle.astype("int64")
df.faelle_per_capita = df.faelle_per_capita.astype("float64")

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   bundesland         17 non-null     object        
 1   faelle             17 non-null     int64         
 2   tod                17 non-null     int64         
 3   neue_faelle        17 non-null     int64         
 4   faelle_per_capita  17 non-null     float64       
 5   lat                17 non-null     float64       
 6   long               17 non-null     float64       
 7   datum              17 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(3), object(1)
memory usage: 1.2+ KB


In [19]:
df

,bundesland,faelle,tod,neue_faelle,faelle_per_capita,lat,long,datum
0,Baden-Württemberg,21603,519,923,195.0,48.50,9.00,2020-04-09
1,Bayern,28827,635,1263,220.0,48.90,11.40,2020-04-09
2,Berlin,4202,37,174,112.0,52.50,13.40,2020-04-09
3,Brandenburg,1578,30,106,63.0,52.00,13.87,2020-04-09
4,Bremen,445,11,21,65.0,53.10,8.80,2020-04-09
5,Hamburg,3320,44,155,180.0,53.50,10.00,2020-04-09
6,Hessen,5242,100,277,84.0,50.70,9.00,2020-04-09
7,Mecklenburg-Vorpommern,572,11,17,36.0,53.80,12.50,2020-04-09
8,Niedersachsen,6804,141,419,85.0,52.57,10.00,2020-04-09
9,Nordrhein-Westfalen,21961,384,1032,122.0,51.60,7.50,2020-04-09


In [20]:
pd.concat([df_old,df]).reset_index().drop("index",axis=1).tail(30)

,bundesland,faelle,tod,neue_faelle,faelle_per_capita,lat,long,datum
623,Thüringen,468,3,+74,22.0,50.85,11.13,2020-03-26
624,Thüringen,542,4,74,25.0,50.85,11.13,2020-03-27
625,Thüringen,584,5,42,27.0,50.85,11.13,2020-03-28
626,Thüringen,697,5,113,33.0,50.85,11.13,2020-03-29
627,Thüringen,719,5,22,34.0,50.85,11.13,2020-03-30
628,Thüringen,784,6,65,37.0,50.85,11.13,2020-03-31
629,Thüringen,860,6,76,40.0,50.85,11.13,2020-04-01
630,Thüringen,925,7,65,43.0,50.85,11.13,2020-04-02
631,Thüringen,982,9,57,46.0,50.85,11.13,2020-04-03
632,Thüringen,1072,10,90,50.0,50.85,11.13,2020-04-04


In [21]:
df=pd.concat([df_old,df]).reset_index().drop("index",axis=1)

In [22]:
df[df["bundesland"]=="Bayern"]

,bundesland,faelle,tod,neue_faelle,faelle_per_capita,lat,long,datum
40,Bayern,15,0,0,0.10,48.9,11.4,2020-02-28
41,Bayern,15,0,0,0.10,48.9,11.4,2020-02-29
42,Bayern,19,0,4,0.10,48.9,11.4,2020-03-01
43,Bayern,25,0,6,0.20,48.9,11.4,2020-03-02
44,Bayern,35,0,10,0.30,48.9,11.4,2020-03-03
45,Bayern,48,0,13,0.40,48.9,11.4,2020-03-04
46,Bayern,52,0,4,0.40,48.9,11.4,2020-03-05
47,Bayern,79,0,27,0.60,48.9,11.4,2020-03-06
48,Bayern,117,0,38,0.90,48.9,11.4,2020-03-07
49,Bayern,148,0,31,1.10,48.9,11.4,2020-03-08


In [23]:
df_old.to_csv ('old_data/brd_covid_19'+str(today)+".csv", index = False, header=True)

In [24]:
df.to_csv ('brd_covid_19.csv', index = False, header=True)